Sklearn Pipeline test

In [2]:
import sklearn
import numpy as np
import pandas as pd
import os.path
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.abspath(''),os.path.pardir)))


In [3]:
from datk.preprocessing import make_column_transformer, get_preprocessed_df




In [4]:

df = pd.read_csv("./train.csv")

df1 = df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
df11 = df.drop(['Name','Ticket','Cabin'],axis=1)

ID = ['PassengerId']

df1.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [5]:
df1.isnull().mean()

Survived    0.000000
Pclass      0.000000
Sex         0.000000
Age         0.198653
SibSp       0.000000
Parch       0.000000
Fare        0.000000
Embarked    0.002245
dtype: float64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [9]:
col = list(df.columns)
col

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [15]:
b = ['Age']

c = set(col) - set(b)
c

{'Cabin',
 'Embarked',
 'Fare',
 'Name',
 'Parch',
 'PassengerId',
 'Pclass',
 'Sex',
 'SibSp',
 'Survived',
 'Ticket'}

In [8]:
df1 = pd.get_dummies(df,dummy_na=True)

In [9]:
df1.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,1,0,3,22.0,1,0,7.2500,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,2,1,1,38.0,1,0,71.2833,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,3,1,3,26.0,0,0,7.9250,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,4,1,1,35.0,1,0,53.1000,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,5,0,3,35.0,0,0,8.0500,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [16]:
from sklearn.pipeline import make_pipeline,Pipeline,FeatureUnion
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer,make_column_selector


num_imputer = Pipeline([
        ("imputer", SimpleImputer(strategy="median",add_indicator=False)),
        
    ])
cat_ohe = Pipeline([
    ("cat_imputer", SimpleImputer(strategy='constant',fill_value='NA')),
    ('ohe',OneHotEncoder(dtype=np.int,handle_unknown='ignore'))
    ])
    

In [17]:
col_transformer = ColumnTransformer(
    [('imp', num_imputer, make_column_selector(dtype_include=np.number)),
     ('ohe', cat_ohe, make_column_selector(dtype_include=['object','category']))
    ], remainder='passthrough'
    )


In [18]:
col_transformer.fit(df1)

ColumnTransformer(remainder='passthrough',
                  transformers=[('imp',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe581b13950>),
                                ('ohe',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant')),
                                                 ('ohe',
                                                  OneHotEncoder(dtype=<class 'int'>,
                                                                handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe581b13890>)])

In [19]:
col_label = col_transformer.transformers_[0][2] + list(col_transformer.transformers_[1][1].steps[1][1].get_feature_names(
    col_transformer.transformers_[1][2]))
col_label

['Survived',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_NA',
 'Embarked_Q',
 'Embarked_S']

In [42]:
df.dtypes
make_column_selector(dtype_include=['object','category'])

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [43]:
np.str

str

In [13]:
df2 = pd.DataFrame(col_transformer.transform(df1),
    columns=col_label, # get column name
    index = df1.index
)

In [14]:
df2.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0


In [16]:
df3 = get_preprocessed_df(df1,col_transformer,fit_flag=False)

df3.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0


In [4]:
col_trans = make_column_transformer()

In [6]:
df4 = get_preprocessed_df(df1,col_trans,fit_flag=True)
df4.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0


In [27]:
df_s = df1.sample(1)
df_s.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
194,1,1,female,44.0,0,0,27.7208,C


In [28]:
df5 = get_preprocessed_df(df_s,col_trans,fit_flag=False)
df5.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S
194,1.0,1.0,44.0,0.0,0.0,27.7208,1.0,0.0,1.0,0.0,0.0,0.0


In [35]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [34]:
df7 = get_preprocessed_df(df,col_trans,fit_flag=False)
df7.head()



,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0


In [29]:
col_trans_smpl = make_column_transformer().fit(df_s)

In [33]:
df6 = get_preprocessed_df(df_s,col_trans_smpl,fit_flag=False)
df6.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Embarked_C
194,1.0,1.0,44.0,0.0,0.0,27.7208,1.0,1.0


In [2]:
col_trans_smpl

NameError: name 'col_trans_smpl' is not defined

In [205]:
import joblib
joblib.dump(col_transformer,'./col_transformer.bin')



['./col_transformer.bin']

In [206]:
ct = joblib.load('./col_transformer.bin')

In [208]:
df3 = pd.DataFrame(ct.transform(df1),
    columns=ct.transformers_[0][2], # get column name
    index = df1.index
)
df3.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [211]:
!rm col_transformer.bin

rm: col_transformer.bin: No such file or directory
Binary_Classifier_Notebook.ipynb  random-forest_classification.yaml
evaluate.py                       random-forest_regression.yaml
example_classification.ipynb      sklearn_pipeline.ipynb
example_regression.ipynb          test.csv
fit                               test_titanic.csv
fit.py                            train.csv
housing.csv                       train_titanic.csv
model_classification.yaml         xgb_classification
model_regression.yaml             xgb_model_classification.yaml
model_results                     xgb_model_regression.yaml
predict.py                        xgb_regression
